In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sql_f
import pandas as pd

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Project_Preprocessing") \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
# read the 2020 dataset
df20 = spark.read.format("csv").option("header", "true").load("/content/drive/MyDrive/CSC590_Project/data/2020/heart_2020_cleaned.csv")

In [ ]:
df20.printSchema()
df20.count()

root
 |-- HeartDisease: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- AlcoholDrinking: string (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- PhysicalHealth: string (nullable = true)
 |-- MentalHealth: string (nullable = true)
 |-- DiffWalking: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Diabetic: string (nullable = true)
 |-- PhysicalActivity: string (nullable = true)
 |-- GenHealth: string (nullable = true)
 |-- SleepTime: string (nullable = true)
 |-- Asthma: string (nullable = true)
 |-- KidneyDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)



319795

In [ ]:
# read the 2022 dataset
df22 = spark.read.format('csv').option("header", "true").load("/content/drive/MyDrive/CSC590_Project/data/2022/heart_2022_no_nans.csv")

In [ ]:
df22.printSchema()
df22.count()

root
 |-- State: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- GeneralHealth: string (nullable = true)
 |-- PhysicalHealthDays: string (nullable = true)
 |-- MentalHealthDays: string (nullable = true)
 |-- LastCheckupTime: string (nullable = true)
 |-- PhysicalActivities: string (nullable = true)
 |-- SleepHours: string (nullable = true)
 |-- RemovedTeeth: string (nullable = true)
 |-- HadHeartAttack: string (nullable = true)
 |-- HadAngina: string (nullable = true)
 |-- HadStroke: string (nullable = true)
 |-- HadAsthma: string (nullable = true)
 |-- HadSkinCancer: string (nullable = true)
 |-- HadCOPD: string (nullable = true)
 |-- HadDepressiveDisorder: string (nullable = true)
 |-- HadKidneyDisease: string (nullable = true)
 |-- HadArthritis: string (nullable = true)
 |-- HadDiabetes: string (nullable = true)
 |-- DeafOrHardOfHearing: string (nullable = true)
 |-- BlindOrVisionDifficulty: string (nullable = true)
 |-- DifficultyConcentrating: string (nullable = t

246022

In [ ]:
# rename equivalent columns in the two dataframes so that they match
df20 = df20.withColumnRenamed("HeartDisease", "HadHeartAttack")\
          .withColumnRenamed("Stroke", "HadStroke")\
          .withColumnRenamed("PhysicalHealth", "PhysicalHealthDays")\
          .withColumnRenamed("MentalHealth", "MentalHealthDays")\
          .withColumnRenamed("DiffWalking", "DifficultyWalking")\
          .withColumnRenamed("GenHealth", "GeneralHealth")\
          .withColumnRenamed("SleepTime", "SleepHours")\
          .withColumnRenamed("Asthma", "HadAsthma")\
          .withColumnRenamed("KidneyDisease", "HadKidneyDisease")\
          .withColumnRenamed("SkinCancer", "HadSkinCancer")
df22 = df22.withColumnRenamed("SmokerStatus", "Smoking")\
          .withColumnRenamed("AlcoholDrinkers", "AlcoholDrinking")\
          .withColumnRenamed("RaceEthnicityCategory", "Race")\
          .withColumnRenamed("HadDiabetes", "Diabetic")\
          .withColumnRenamed("PhysicalActivities", "PhysicalActivity")

In [ ]:
# generate a set of common columns
common_cols = set(df20.columns).intersection(df22.columns)
common_cols

{'AgeCategory',
 'AlcoholDrinking',
 'BMI',
 'Diabetic',
 'DifficultyWalking',
 'GeneralHealth',
 'HadAsthma',
 'HadHeartAttack',
 'HadKidneyDisease',
 'HadSkinCancer',
 'HadStroke',
 'MentalHealthDays',
 'PhysicalActivity',
 'PhysicalHealthDays',
 'Race',
 'Sex',
 'SleepHours',
 'Smoking'}

In [ ]:
# check that all columns in df20 are now in both datasets
print(set(df20.columns) == common_cols)

True


In [ ]:
# appropriately re-cast columns
df20 = df20.withColumn("BMI", df20["BMI"].cast("float"))\
          .withColumn("PhysicalHealthDays", df20["PhysicalHealthDays"].cast("int"))\
          .withColumn("MentalHealthDays", df20["MentalHealthDays"].cast("int"))\
          .withColumn("SleepHours", df20["SleepHours"].cast("int"))
df22 = df22.withColumn("BMI", df22["BMI"].cast("float"))\
          .withColumn("PhysicalHealthDays", df22["PhysicalHealthDays"].cast("int"))\
          .withColumn("MentalHealthDays", df22["MentalHealthDays"].cast("int"))\
          .withColumn("SleepHours", df22["SleepHours"].cast("int"))\
          .withColumn("HeightInMeters", df22["HeightInMeters"].cast("float"))\
          .withColumn("WeightInKilograms", df22["WeightInKilograms"].cast("float"))

In [ ]:
# compare common columns in df1 and df2
# if the column is numeric, desribe it
# if the column is not numeric, compare the unique values in the df1 column and the df2 column
# if the unique values differ, flag the column- that will need to be addressed

def compare_dataframes(df1, df2):

  pdf1 = df1.toPandas()
  pdf2 = df2.toPandas()

  # find the columns present in both dataframes
  common_cols = set(pdf1.columns).intersection(pdf2.columns)

  for col in common_cols:
    print(f"--- Column: {col} ---")

    # numeric columns
    if pd.api.types.is_numeric_dtype(pdf1[col]):
      print("df1:")
      print(pdf1[col].describe())

      print("df2:")
      print(pdf2[col].describe())

    # non-numeric columns
    else:
      print("df1:")
      unique_vals_1 = set(pdf1[col].unique())
      print(unique_vals_1)

      print("df2:")
      unique_vals_2 = set(pdf2[col].unique())
      print(unique_vals_2)

      # unique values differ, flag column
      if unique_vals_1 != unique_vals_2:
        print(f"***DIFFERENT VALUES***")
        print(f"Unique to df1: {unique_vals_1 - unique_vals_2}")
        print(f"Unique to df2: {unique_vals_2 - unique_vals_1}")

    print()

compare_dataframes(df20, df22)

--- Column: HadKidneyDisease ---
df1:
{'Yes', 'No'}
df2:
{'Yes', 'No'}

--- Column: HadHeartAttack ---
df1:
{'Yes', 'No'}
df2:
{'Yes', 'No'}

--- Column: Sex ---
df1:
{'Male', 'Female'}
df2:
{'Male', 'Female'}

--- Column: AgeCategory ---
df1:
{'35-39', '25-29', '70-74', '55-59', '75-79', '60-64', '65-69', '80 or older', '18-24', '40-44', '30-34', '45-49', '50-54'}
df2:
{'Age 45 to 49', 'Age 35 to 39', 'Age 40 to 44', 'Age 25 to 29', 'Age 18 to 24', 'Age 65 to 69', 'Age 50 to 54', 'Age 30 to 34', 'Age 75 to 79', 'Age 80 or older', 'Age 70 to 74', 'Age 55 to 59', 'Age 60 to 64'}
***DIFFERENT VALUES***
Unique to df1: {'35-39', '25-29', '70-74', '55-59', '75-79', '60-64', '65-69', '80 or older', '18-24', '40-44', '30-34', '45-49', '50-54'}
Unique to df2: {'Age 45 to 49', 'Age 25 to 29', 'Age 40 to 44', 'Age 18 to 24', 'Age 35 to 39', 'Age 65 to 69', 'Age 50 to 54', 'Age 30 to 34', 'Age 75 to 79', 'Age 80 or older', 'Age 70 to 74', 'Age 55 to 59', 'Age 60 to 64'}

--- Column: AlcoholDrinki

In [ ]:
# replace values in df22 that differ from those in df20

df22 = df22.withColumn("Smoking", sql_f.when((df22["Smoking"].startswith("Current") | df22["Smoking"].startswith("Former")), "Yes")\
                       .when(df22["Smoking"] == "Never smoked", "No")\
                       .otherwise(df22["Smoking"]))

df22 = df22.withColumn("Diabetic", sql_f.when(df22["Diabetic"] == "No, pre-diabetes or borderline diabetes", "No, borderline diabetes")\
                       .when(df22["Diabetic"] == "Yes, but only during pregnancy (female)", "Yes (during pregnancy)")\
                       .otherwise(df22["Diabetic"]))

df22 = df22.withColumn("AgeCategory", sql_f.regexp_replace("AgeCategory", "Age ", ""))
df22 = df22.withColumn("AgeCategory", sql_f.regexp_replace("AgeCategory", " to ", "-"))

df22 = df22.withColumn("Race", sql_f.when(df22["Race"].startswith("Multiracial"), "Other")\
                       .when(df22["Race"].startswith("Black"), "Black")\
                       .when(df22["Race"].startswith("White"), "White")\
                       .when(df22["Race"].startswith("Other"), "Other")\
                       .otherwise(df22["Race"]))

In [ ]:
compare_dataframes(df20, df22)

--- Column: HadKidneyDisease ---
df1:
{'Yes', 'No'}
df2:
{'Yes', 'No'}

--- Column: HadHeartAttack ---
df1:
{'Yes', 'No'}
df2:
{'Yes', 'No'}

--- Column: Sex ---
df1:
{'Male', 'Female'}
df2:
{'Male', 'Female'}

--- Column: AgeCategory ---
df1:
{'35-39', '25-29', '70-74', '55-59', '75-79', '60-64', '65-69', '80 or older', '18-24', '40-44', '30-34', '45-49', '50-54'}
df2:
{'35-39', '25-29', '70-74', '75-79', '60-64', '55-59', '65-69', '80 or older', '18-24', '40-44', '30-34', '45-49', '50-54'}

--- Column: AlcoholDrinking ---
df1:
{'Yes', 'No'}
df2:
{'Yes', 'No'}

--- Column: SleepHours ---
df1:
count    319795.000000
mean          7.097075
std           1.436007
min           1.000000
25%           6.000000
50%           7.000000
75%           8.000000
max          24.000000
Name: SleepHours, dtype: float64
df2:
count    246022.000000
mean          7.021331
std           1.440681
min           1.000000
25%           6.000000
50%           7.000000
75%           8.000000
max          24.

In [ ]:
# combine the two dataframes
df20_common = df20.select(list(common_cols))
df22_common = df22.select(list(common_cols))

df = df20_common.union(df22_common)

df.printSchema()
print(set(df.columns) == common_cols == set(df20.columns))
df.count()

root
 |-- HadKidneyDisease: string (nullable = true)
 |-- HadHeartAttack: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- AlcoholDrinking: string (nullable = true)
 |-- SleepHours: integer (nullable = true)
 |-- DifficultyWalking: string (nullable = true)
 |-- HadAsthma: string (nullable = true)
 |-- HadSkinCancer: string (nullable = true)
 |-- Diabetic: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- BMI: float (nullable = true)
 |-- PhysicalHealthDays: integer (nullable = true)
 |-- Race: string (nullable = true)
 |-- PhysicalActivity: string (nullable = true)
 |-- HadStroke: string (nullable = true)
 |-- MentalHealthDays: integer (nullable = true)
 |-- GeneralHealth: string (nullable = true)

True


565817

In [ ]:
# save the new dataset
df.write.format("csv").option("header", "true").save("/content/drive/MyDrive/CSC590_Project/data/combined_data.csv")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/drive/MyDrive/CSC590_Project/data/combined_data.csv already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
# Further cleaning and integration of heart_2020 and heart_2022 datasets

# Standardize column names to lowercase and remove whitespace/special characters
def standardize_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)
    return df

# Standardize column names for both datasets
heart_2020 = standardize_columns(heart_2020)
heart_2022 = standardize_columns(heart_2022)

# Find common columns
common_columns = heart_2020.columns.intersection(heart_2022.columns)

# Extract only common features
heart_2020_common = heart_2020[common_columns]
heart_2022_common = heart_2022[common_columns]

# Concatenate datasets
integrated_df = pd.concat([heart_2020_common, heart_2022_common], ignore_index=True, sort=False)

# Save the integrated dataset for further processing
integrated_df.to_csv('/content/drive/MyDrive/CSC590_Project/data/integrated_heart_data.csv', index=False)

# Display dataset summary
print("Integrated Dataset Info:")
print(integrated_df.info())
print("\nPreview of Integrated Dataset:")
print(integrated_df.head())


NameError: name 'heart_2020' is not defined